In [1]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import scipy as sp
import coord_transform as coord
import time
from scipy.spatial import distance
from matplotlib import pyplot as plt
import numpy as np
from sklearn.mixture import GaussianMixture
from dask import delayed
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster# this is experimental
from dask_cuda import LocalCUDACluster
from dask.dataframe import from_pandas
import dask.array as da
import dask.bag as db
import cupy as cp
from numba import cuda
from numba import roc
from numba import njit
import numba as nb
import dask
import dask_cudf
import gc
import cupy
import cudf
import os
import time
import math
from multiprocessing import Pool
from itertools import product

In [2]:
def prepare_data(df, hosts=False):
    # Reduce data
    if not hosts:
        df = df[4.5 > df["parallax"]/df["parallax_error"]]
    
    # Calculate distance in pc
    df["distance_pc"] = 1./df["parallax"]
    df = df[df["distance_pc"]>0]
    
    # Convert from spherical to cartesian coordinates
    df["x"], df["y"], df["z"] = coord.sph2cart(df["distance_pc"], df["ra"], df["dec"])
    df["vx"],df["vy"],df["vz"] = coord.vsph2cart(df["dr2_radial_velocity"], df["pmra"], df["pmdec"], df["distance_pc"], df["ra"], df["dec"])
    
    # Remove unnecessary columns
    if hosts:
        df = df.drop(df.columns[1:24], axis=1)
    else:
        df = df.drop(df.columns[:13], axis=1)
    
    return df

# Load exoplanet host stars data

In [3]:
hosts = cudf.read_csv("hosts.csv")

In [4]:
hosts = prepare_data(hosts, True)

# Load gaia stars dataset

In [5]:
gaia = cudf.read_csv("gaia.csv")

In [6]:
gaia = prepare_data(gaia)

In [7]:
hosts = hosts.to_pandas()

In [8]:
gaia = gaia.to_pandas().to_numpy()

In [9]:
@njit
def to_sets(target):
    dist2 = 80
    
    set2 = []
    for j in range(gaia.shape[0]):
        z = np.zeros(shape=target.shape)
        for i in range(len(target)):
            z[i] = target[i] - gaia[j][i+1]
        
        dist = np.sqrt(np.sum(z**2, 0))

        if dist < dist2:
            set2.append([gaia[j][1], gaia[j][2], gaia[j][3], gaia[j][4], gaia[j][5], gaia[j][6]])

    return np.array(set2)

In [10]:
@njit
def calc_mah(i, set2, set2_inv):

    mahal_dist = np.full(20, np.inf, dtype="float64")
    
    for j in range(0, set2.shape[0]):
        set2_inv = np.atleast_2d(set2_inv)
        delta = set2[i] - set2[j]
        m = np.dot(np.dot(delta, set2_inv), delta)

        if np.sqrt(m) < mahal_dist.max():
            mahal_dist[mahal_dist.argmax()] = np.sqrt(m)
        
    return mahal_dist.max()

### Convert mahalanobis distances of all stars in set1 to density

In [11]:
@njit
def calc_dense(mah_dist_arr):
    density = 20/(mah_dist_arr)**6

    norm_density = density/np.median(density)
    return norm_density

### Use density to create GausianMixture model and draw low and high density groups

In [12]:
def draw(norm_density):
    # Add second dimension and transpose array to match input format for GaussianMixture
    X = np.expand_dims(np.log10(norm_density), axis=0).T

    # train GaussianMixture
    model = GaussianMixture(2).fit(X)

    # Define figure params
    fig = plt.figure(figsize=(50, 5))
    fig.subplots_adjust(left=0.12, right=0.97, bottom=0.21, top=0.9, wspace=0.5)
    ax = fig.add_subplot(131)

    x = np.linspace(-6, 6, 1000)
    logprob = model.score_samples(x.reshape(-1, 1))
    responsibilities = model.predict_proba(x.reshape(-1, 1))
    pdf = np.exp(logprob)
    pdf_individual = responsibilities * pdf[:, np.newaxis]

    ax.hist(X, 30, density=True, histtype='stepfilled', alpha=0.4)
    # Add combined kde line
    ax.plot(x, pdf, '-k')
    # Add individual lines for low and high density
    ax.plot(x, pdf_individual, '--k')
    ax.text(0.04, 0.96, "Best-fit Mixture", ha='left', va='top', transform=ax.transAxes)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$p(x)$')
    plt.savefig("plot_v1.png", bbox_inches='tight')
    plt.show()
    
    return model

### Loop through all planets and apply above functions. 
### For testing purposes I am using only 3 planets and 1000 stars from set1. This takes around 90 seconds on my PC to compute. At the moment for planet with 150000 neighbours within 40pc it would take around 1 hour and 10 minutes to compute.

In [142]:
def get_stars():

    o=0
    density_list = []
    model = 0
    names = np.chararray((hosts.shape[0], 1), itemsize=20)
    names = []
    mah_dist_arr = np.zeros((hosts.shape[0], 1))
    for i, row in hosts.iterrows():
        target = np.array([row[2], row[3], row[4]])

        set2 = to_sets(target)

        if set2.shape[0] < 20:
            continue

        set2_cov_mat = np.cov(set2.T) # Calculate the covariance matrix
        set2_inv = np.linalg.inv(set2_cov_mat)

        names.append(row["hostname"])
        mah_dist_arr[i] = calc_mah(i, set2, set2_inv)

        o = o + 1
    df = pd.DataFrame(mah_dist_arr, index=names, columns=["val"])
    return df

In [143]:
start = time.perf_counter()
mah_dist_arr = get_stars()
end = time.perf_counter()
end-start

117.59713178399943

In [186]:
density = 20/(mah_dist_arr["val"])**6
norm_density = density/np.median(density)

In [188]:
X = np.expand_dims(np.log10(norm_density), axis=0).T

In [193]:
df = pd.DataFrame(X, index=mah_dist_arr.index, columns=["density"])

In [195]:
df.to_csv("planets_dense_6d.csv")